In [1]:
import time
from web3 import Web3, constants
import json
import requests
from config import *
import math
from datetime import date
from decouple import config

from network_utils import *
from math_utils  import *

In [2]:
INFURA_KEY = config('INFURA_KEY')
infura_url = f"https://arbitrum-mainnet.infura.io/v3/{INFURA_KEY}"

web3 = Web3(Web3.HTTPProvider(infura_url))

In [3]:
import time
import datetime

timestamp = time.mktime(datetime.datetime.strptime("24/05/2023", "%d/%m/%Y").timetuple())

def get_block_by_timestamp(timestamp):
    myApiKeyToken = "17HC99ZNQ6NT9I35F695WYT31J2WUE7UF2"
    res = requests.get(f'https://api.arbiscan.io/api?module=block&action=getblocknobytime&timestamp={timestamp}&closest=before&apikey={myApiKeyToken}')
    response = json.loads(res.text)['result']
    return int(response)
block_number = get_block_by_timestamp(int(timestamp))

In [4]:
with open(r"C:\Users\vgtgd\Pendle\PythonWeb3\GLP-28MARCH2024.json", 'r') as f:
  GLP = json.load(f)

GLP

{'name': 'SY GLP',
 'SY': '0x2066a650AF4b6895f72E618587Aad5e8120B7790',
 'YT': '0x56051f8e46b67b4d286454995dbc6f5f3c433e34',
 'PT': '0x96015d0fb97139567a9ba675951816a0bb719e3c',
 'market': '0x7D49E5Adc0EAAD9C027857767638613253eF125f',
 'expiry': 1711584000,
 'scalarRoot': '10484880000000000000',
 'initAnchor': '1285295000000000000'}

In [5]:
YT_contract = get_contract_arb(Web3.to_checksum_address(GLP["YT"]))

In [6]:
index = web3.from_wei(YT_contract.functions.pyIndexStored().call(block_identifier=block_number), 'ether')
index/2

Decimal('0.5')

In [8]:
# YT_abi = requests.get('https://api.arbiscan.io/api?module=contract&action=getabi&address=0x23231848ec8acD3a108C6a61049FbC25A697b594&to_normal_float=raw').text
# YT_address = GLP["YT"]

YT_contract = get_contract_arb(Web3.to_checksum_address(GLP["YT"]))
index = web3.from_wei(YT_contract.functions.pyIndexStored().call(block_identifier=block_number), 'ether')

time.sleep(10)
market_contract = get_contract_arb(address=GLP["market"])

time.sleep(10)
SY_contract = get_contract_arb(address=GLP["SY"])

In [10]:
def get_info(SY_contract, YT_contract, market_contract, timestamp):
    date = datetime.datetime.fromtimestamp(timestamp).strftime('%d-%m-%y')
    block_number = int(get_block_by_timestamp(int(timestamp)))
    PENDLE_reward_index = to_normal_float(market_contract.functions.rewardState(PENDLE_ADDRESS_ARB).call(block_identifier=block_number)[0])

    X_reward_index = to_normal_float(
        SY_contract.functions.rewardIndexesCurrent().call(block_identifier=block_number)[0]
    )

    SY_index = to_normal_float(
        YT_contract.functions.pyIndexStored().call(block_identifier=block_number)
    )

    market_state_values = market_contract.functions.readState(constants.ADDRESS_ZERO).call(block_identifier=block_number)

    market_state_keys = [
        "totalPt",
        "totalSy",
        "totalLp",
        "treasury",
        "scalarRoot",
        "expiry",
        "lnFeeRateRoot",
        "reserveFeePercent",
        "lastLnImpliedRate"
    ]
    #     struct MarketState {
    #     int256 totalPt;
    #     int256 totalSy;
    #     int256 totalLp;
    #     address treasury;
    #     /// immutable variables ///
    #     int256 scalarRoot;
    #     uint256 expiry;
    #     /// fee data ///
    #     uint256 lnFeeRateRoot;
    #     uint256 reserveFeePercent; // base 100
    #     /// last trade data ///
    #     uint256 lastLnImpliedRate;
    # }
    market_state = {market_state_keys[i]: market_state_values[i] for i in range(len(market_state_keys))}
    market_precompute = get_market_precompute(market_state, SY_index, timestamp)

    # time_to_expiry = market_state["expiry"] - timestamp

    data_onchain = {}
    data_onchain["date"] = date
    data_onchain["asset"] = market_precompute["total_asset"]
    data_onchain["PT"] = to_normal_float(market_state["totalPt"])
    data_onchain["LP"] = to_normal_float(market_state["totalLp"])
    data_onchain["exchange_rate"] = get_exchange_rate(data_onchain["PT"], data_onchain["asset"], market_precompute["rate_scalar"], market_precompute["rate_anchor"])
    data_onchain["X_reward_index"] = X_reward_index
    data_onchain["PENDLE_reward_index"] = PENDLE_reward_index
    data_onchain["average_boost_factor"] = average_boost_factor(market_contract, block_number)
    
    return data_onchain

data_onchain = get_info(SY_contract, YT_contract, market_contract, timestamp)
data_onchain


{'date': '24-05-23',
 'asset': 7723058.486487405,
 'PT': 3434978.8982548523,
 'LP': 5707057.177257043,
 'exchange_rate': 1.1516421065224196,
 'X_reward_index': 2.91574597944e-05,
 'PENDLE_reward_index': 0.5049281080968737,
 'ETH_price': 1893.51327570915,
 'average_boost_factor': 1.1763815114857266}

In [9]:
import pandas as pd

data_df = pd.DataFrame()

timestamp_start = int(time.mktime(datetime.datetime.strptime("24/02/2023", "%d/%m/%Y").timetuple()))
timestamp_end = int(time.mktime(datetime.datetime.strptime("08/06/2023", "%d/%m/%Y").timetuple()))

for i in range(timestamp_start, timestamp_end, DAY_DELTA * DAY):
    data_onchain = pd.DataFrame(get_info(SY_contract, YT_contract, market_contract, i), index=[0])
    data_df = pd.concat([data_df, data_onchain], ignore_index=True)

In [10]:
def IL(data_df, case, PENDLE_incentive = False):
    n = len(data_df)
    IL = [[0 for _ in range(n)] for _ in range(n)]

# Compare from timestamp i to timestamp j
    for i in range(n):
        for j in range(i, n):             
            BOOST_FACTOR = data_df.loc[j, "average_boost_factor"]

            WETH_reward_value_each_SY = (data_df.loc[j, "X_reward_index"] - data_df.loc[i, "X_reward_index"])*data_df.loc[j, "ETH_price"]
            
            # At timestamp j
            PT_in_pool_value = data_df.loc[j, "PT"]/(data_df.loc[j, "exchange_rate"]*data_df.loc[j, "LP"]) # value unit is SY
            SY_in_pool = data_df.loc[j, "asset"]/data_df.loc[j, "LP"]
            WETH_in_pool_reward_value = SY_in_pool*WETH_reward_value_each_SY*BOOST_FACTOR # Assume receive reward based on the original SY amount
            PENDLE_reward_value = (data_df.loc[j, "PENDLE_reward_index"] - data_df.loc[i, "PENDLE_reward_index"])*BOOST_FACTOR*PENDLEGLP

            in_pool_value = (PT_in_pool_value + SY_in_pool + WETH_in_pool_reward_value) + (PENDLE_reward_value if PENDLE_incentive else 0)
            
            if case == 'A':  # User holds PT + SY
                PT_out_pool_value = data_df.loc[i, "PT"]/(data_df.loc[j, "exchange_rate"]*data_df.loc[i, "LP"])
                SY_out_pool = data_df.loc[i, "asset"]/data_df.loc[i, "LP"]
                WETH_out_pool_reward_value = SY_out_pool*WETH_reward_value_each_SY
                out_pool_value =  PT_out_pool_value + SY_out_pool + WETH_out_pool_reward_value
            
            elif case == 'B': # User holds SY only
                SY_out_pool = (data_df.loc[i, "PT"]/data_df.loc[i, "exchange_rate"] + data_df.loc[i, "asset"])/data_df.loc[i, "LP"]
                WETH_out_pool_reward_value = SY_out_pool*WETH_reward_value_each_SY
                out_pool_value =  SY_out_pool + WETH_out_pool_reward_value
            
            elif case =='C': 
                YT_holding_amount = data_df.loc[i, "PT"]/data_df.loc[i, "LP"]
                YT_value = YT_holding_amount*(1-1/data_df.loc[j, "exchange_rate"])
                WETH_in_pool_reward_value = WETH_in_pool_reward_value + YT_holding_amount*YT_REWARD_FACTOR*WETH_reward_value_each_SY
                in_pool_value = (PT_in_pool_value + SY_in_pool + WETH_in_pool_reward_value + YT_value) + (PENDLE_reward_value if PENDLE_incentive else 0)

                SY_out_pool = (data_df.loc[i, "PT"] + data_df.loc[i, "asset"])/data_df.loc[i, "LP"]
                WETH_out_pool_reward_value = SY_out_pool*WETH_reward_value_each_SY
                out_pool_value = SY_out_pool + WETH_out_pool_reward_value

            IL[i][j] = in_pool_value/out_pool_value

    return IL


In [11]:
IL_A1 = IL(data_df, 'A', False)
IL_A2 = IL(data_df, 'A', True)
IL_B1 = IL(data_df, 'B', False)
IL_B2 = IL(data_df, 'B', True)
IL_C1 = IL(data_df, 'C', False)
IL_C2 = IL(data_df, 'C', True)

In [12]:
def write_array_to_csv(array, file_name):
    date_column = [date.fromtimestamp(timestamp_start) + datetime.timedelta(days=DAY_DELTA*i) for i in range(len(array))]
    for i in range(len(array)):
        array[i].insert(0, date_column[i])

    date_row = [''] + date_column
    array.insert(0, date_row)

    df = pd.DataFrame(array)
    df.to_csv(file_name, header=False, index=False)

In [13]:
write_array_to_csv(IL_A1, "data/IL_A1.csv")
write_array_to_csv(IL_A2, "data/IL_A2.csv")
write_array_to_csv(IL_B1, "data/IL_B1.csv")
write_array_to_csv(IL_B2, "data/IL_B2.csv")
write_array_to_csv(IL_C1, "data/IL_C1.csv")
write_array_to_csv(IL_C2, "data/IL_C2.csv")

In [14]:
SYamount =  106.009646748417638518
block_number_start = 70169650
block_number_end = 102482129

X_reward_index_start = web3.from_wei(
        SY_contract.functions.rewardIndexesCurrent().call(block_identifier=block_number_start)[0],
        'ether'
    )

X_reward_index_end = web3.from_wei(
        SY_contract.functions.rewardIndexesCurrent().call(block_identifier=block_number_end)[0],
        'ether'
)
# print(X_reward_index_end, X_reward_index_start)
WETH_reward = SYamount*float(X_reward_index_end - X_reward_index_start)
WETH_reward

0.0028218324686607376